In [26]:
import torch
import torch.nn as nn
import torch.optim  as optim
from PIL import Image
import torchvision.transforms as tt
import torchvision.models as models
from torchvision.utils import save_image

In [27]:
model = models.vgg19(pretrained=True).features
model

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [32]:
# ['0', '5', '10', '19', '28']
import matplotlib.pyplot as plt

noise = torch.randn([356, 356], requires_grad =  True).detach().numpy()
noise_2 = torch.randn([356, 356], requires_grad =  True)
# noise = torch.randn(3, 200, 200, requires_grad = True)
# plt.imshow(noise)

gented = orig_img.clone().requires_grad_(True)
print(type(noise))
print(gented.shape)

<class 'numpy.ndarray'>
torch.Size([1, 3, 356, 356])


In [29]:
class VGG(nn.Module):
  def __init__(self):
    super(VGG, self).__init__()

    self.chosen_feature = ['0', '5', '10', '19', '28']
    self.model = models.vgg19(pretrained=True).features[:29]

  def forward(self, x):
    features = []

    for layer_num, layer in enumerate(self.model):
      x = layer(x)

      if str(layer_num) in self.chosen_feature:
        features.append(x)


    return features

def load_image(image_name):
  
  image = Image.open(image_name)
  image = loader(image).unsqueeze(0)
  
  return image.to(device)

device = torch.device('cuda' if torch.cuda.is_available else "cpu")
image_size = 356

loader = tt.Compose([tt.Resize((image_size, image_size)),
                            tt.ToTensor(),
                            # tt.Normalize()
                            ])



In [ ]:

orig_img = load_image('alona-tal.jpg')
style_image = load_image('style.jpg')

model = VGG().to(device).eval()

# gented = orig_img.clone().requires_grad_(True)
gented = torch.randn([1,3,image_size, image_size], requires_grad =  True, device = "cuda")

tot_steps = 5000
lr = 0.001
alpha = 1
beta = 0.01
optimizer = optim.Adam([gented], lr = lr)

for step in range(tot_steps):
  gen_feature = model(gented)
  orig_img_feature = model(orig_img)
  style_feature = model(style_image)

  style_loss = original_loss = 0

  for gene_feature, orig_feature, style_feat in zip(gen_feature, orig_img_feature, style_feature):
    
      batch_size, channel, height, width = gene_feature.shape
      original_loss += torch.mean((gene_feature - orig_feature)**2)


      #GRAM MATRIX
      g = gene_feature.view(channel, height*width).mm(
          gene_feature.view(channel, height*width).t()
      )

      A = style_feat.view(channel, height*width).mm(
          style_feat.view(channel, height*width).t()
      )

      style_loss += torch.mean((g-A)**2)

  total_loss = alpha*original_loss + beta*style_loss
  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()

  if step%200 ==0:
    print(total_loss)
    save_image(gented, "generated.jpg")


tensor(5.4942e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.8482e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(98093696., device='cuda:0', grad_fn=<AddBackward0>)
tensor(63154960., device='cuda:0', grad_fn=<AddBackward0>)
tensor(45412552., device='cuda:0', grad_fn=<AddBackward0>)
tensor(34919248., device='cuda:0', grad_fn=<AddBackward0>)
tensor(27613510., device='cuda:0', grad_fn=<AddBackward0>)
tensor(22060446., device='cuda:0', grad_fn=<AddBackward0>)
tensor(17789366., device='cuda:0', grad_fn=<AddBackward0>)
tensor(14595684., device='cuda:0', grad_fn=<AddBackward0>)
tensor(12192102., device='cuda:0', grad_fn=<AddBackward0>)
tensor(10309004., device='cuda:0', grad_fn=<AddBackward0>)
tensor(8797890., device='cuda:0', grad_fn=<AddBackward0>)
tensor(7577359., device='cuda:0', grad_fn=<AddBackward0>)
tensor(6577559.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5744880., device='cuda:0', grad_fn=<AddBackward0>)
tensor(5037869., device='cuda:0', grad_fn=<AddBackward